In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_ollama import ChatOllama

In [2]:
# import pandas as pd 

# chunk_metadata = pd.read_csv('chunks.csv')
# chunk_metadata['text'][561]

In [3]:
import os
# Force GPU usage
os.environ["OLLAMA_ACCELERATOR"] = "gpu"

chat_model = ChatOllama(
    model="gemma3:1b",
    num_predict = 512,
    temperature=0.2,
    top_k=10,
    additional_kwargs = {'gpu': True})

In [4]:
label_template = """
Bạn là trợ lý AI được giao nhiệm vụ tạo ra một cặp câu hỏi-trả lời thực tế duy nhất dựa trên một vài tài liệu nhất định về pháp luật Việt Nam, về luật an ninh mạng và an toàn thông tin mạng, các sản phẩm về công nghệ thông tin, mật mã, dữ liệu người dùng. 
Câu hỏi phải là thứ mà người dùng có thể tự nhiên hỏi khi tìm kiếm thông tin có trong tài liệu.
Với tài liệu: {chunk}

Hãy tuân theo các yêu cầu sau khi tạo câu hỏi - câu trả lời:

1. Phân tích các chủ đề, sự kiện và quy định quan trọng trong tài liệu đã cho, chọn một chủ đề cụ thể liên quan đến luật an ninh mạng hoặc an toàn thông tin mạng.
2. Tạo ra một câu hỏi tự nhiên mà người dùng có thể hỏi để tìm kiếm thông tin trong tài liệu này.
3. Sử dụng ngôn ngữ tự nhiên, có thể bao gồm lỗi đánh máy nhỏ hoặc cách diễn đạt thông thường của người dùng khi tìm kiếm.
4. Đảm bảo câu hỏi có liên quan về mặt ngữ nghĩa với nội dung tài liệu MÀ KHÔNG sao chép trực tiếp các cụm từ.
5. Đảm bảo rằng tất cả các câu hỏi đều tập trung vào một chủ đề tương tự hoặc yêu cầu cùng một dạng thông tin.
6. Đảm bảo câu hỏi và câu trả lời được viết bằng tiếng Việt.
7: Đảm bảo rằng đầu ra phải là một JSON hợp lệ chứa cả 'question' và 'answer'. Nếu đầu ra không chứa cả 'question' và 'answer', kết quả sẽ bị coi là không hợp lệ.

Trả về một đối tượng JSON có cấu trúc như sau:
{{
  "question": "Văn bản câu hỏi đã tạo",
  "answer": "Văn bản câu trả lời đã tạo"
}}

LƯU Ý QUAN TRỌNG: Chỉ trả về đối tượng JSON, không thêm bất kỳ văn bản nào khác. Không thêm ghi chú, giải thích, hoặc bất kỳ thông tin nào ngoài JSON.
"""

label_prompt = ChatPromptTemplate.from_template(label_template, stream = False)

label_chain = label_prompt | chat_model | JsonOutputParser()
# label_chain = label_prompt | chat_model

In [5]:
import pandas as pd
from tqdm.auto import tqdm
import random

# Define the number of pairs per chunk
min_pairs = 2
max_pairs = 3

max_try = 5

# Initialize an empty list to store the pairs
pairs = []

error_chunks = []

chunk_metadata = pd.read_csv('data/chunks.csv')
# Iterate over the chunks
for i in tqdm(range(len(chunk_metadata))):
    # Invoke the chain to generate the pairs
    chunk = chunk_metadata['content'][i]
    cid = chunk_metadata['cid'][i]
    num_pairs = random.randint(min_pairs, max_pairs)

    for _ in range(num_pairs):
        for _ in range(max_try):
            try:
                result = label_chain.invoke({"chunk": chunk})
                # Extract the question and answer from the result
                question = result['question']
                answer = result['answer']
                # Create a dictionary with the question, answer, and corpus id
                pair = {'question': question, 'answer': answer, 'cid': cid, 'corpus': chunk}
                # Append the pair to the list
                pairs.append(pair)
                break
            except Exception as e:
                print(f"Error: {e}")
        else:
            error_chunks.append(cid)

# Create a pandas DataFrame from the list of pairs
df = pd.DataFrame(pairs)
df.to_csv('data/train_data.csv', index=False)

/home/quangminh/anaconda3/envs/chatbots/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 12/4142 [00:57<4:35:12,  4.00s/it]

Error: Invalid json output: ```json
{
 "question": "Các quy định pháp luật về bảo vệ dữ liệu cá nhân trong lĩnh vực công nghệ thông tin có thể được áp dụng như thế nào để ngăn chặn các hành vi lạm dụng, khai thác, hoặc sử dụng dữ liệu cá nhân trái phép?",
 "answer": "Luật Bảo vệ Dữ liệu Cá nhân (Luật رقم 20/2023/DL) quy định các biện pháp bảo vệ dữ liệu cá nhân, bao gồm cả việc xác thực, xử lý, lưu trữ, truyền tải, và bảo mật dữ liệu. Việc sử dụng dữ liệu cá nhân cần tuân thủ các quy định về quyền riêng tư, an toàn, và trách nhiệm giải trình. Các biện pháp này bao gồm việc thiết lập các chính sách bảo mật, kiểm soát truy cập dữ liệu, và xử lý dữ liệu theo nguyên tắc "chỉ khi cần thiết, chỉ cho đến khi cần thiết, và không dùng cho mục đích khác".  Việc vi phạm các quy định này có thể bị xử lý theo các hình phạt nghiêm trọng, bao gồm phạt tiền và truy tố hình sự."
}
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


  7%|▋         | 284/4142 [19:44<5:09:21,  4.81s/it]

Error: Invalid json output: ```json
{
 "question": "Luật bảo vệ dữ liệu cá nhân trong lĩnh vực công nghệ thông tin là gì, và những biện pháp nào được áp dụng để đảm bảo quyền riêng tư của người dùng?",
 "answer": "Luật bảo vệ dữ liệu cá nhân trong lĩnh vực công nghệ thông tin (Luật An toàn Thông tin) quy định về việc bảo vệ dữ liệu cá nhân của người dùng, đặc biệt là dữ liệu nhạy cảm như thông tin cá nhân, thông tin tài chính, thông tin sức khỏe. Các biện pháp chính bao gồm: (1) Quyền được bảo vệ dữ liệu; (2) Quyền truy cập, sửa chữa, yêu cầu xóa dữ liệu; (3) Quyền hạn chế xử lý dữ liệu; (4) Bồi thường thiệt hại; (5) Trách nhiệm của tổ chức xử lý dữ liệu; (6) Xử lý dữ liệu theo mục đích, phương thức hợp pháp; (7) Đảm bảo an ninh, bảo mật dữ liệu; (8) Xử lý dữ liệu theo nguyên tắc "chỉ khi cần thiết" và "tính chất hạn chế". Các tổ chức như Bộ Thông tin và Truyền thông, các công ty viễn thông, và các cơ quan quản lý có trách nhiệm thực thi luật này."
}
```
For troubleshooting, visit: htt

  7%|▋         | 286/4142 [20:10<9:04:24,  8.47s/it]

...

 81%|████████  | 3343/4142 [4:11:25<54:19,  4.08s/it]  

Error: Invalid json output: ```json
{
 "question": "Theo quy định về bảo vệ dữ liệu cá nhân, điều gì là quan trọng nhất trong việc thiết kế và triển khai hệ thống thu thập và xử lý dữ liệu người dùng?",
 "answer": "Luật bảo vệ dữ liệu cá nhân quy định rằng dữ liệu cá nhân phải được bảo vệ theo nguyên tắc "chỉ sử dụng, chỉ lưu trữ, chỉ xử lý và chỉ truyền tải dữ liệu cho những người có quyền theo luật pháp." Điều này bao gồm việc đảm bảo dữ liệu được bảo vệ an toàn, được lưu trữ an toàn, được sử dụng một cách hợp lý và chỉ được truyền tải cho những người có quyền. Việc thiết kế hệ thống thu thập và xử lý dữ liệu phải tuân thủ các quy định này để đảm bảo quyền lợi của người dùng và bảo vệ tính riêng tư."
}
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


 92%|█████████▏| 3800/4142 [4:42:21<35:12,  6.18s/it]  

Error: Invalid json output: ```json
{
 "question": "Phân tích quy định về bảo vệ dữ liệu người dùng trong công ty cổ phần, đặc biệt là về trách nhiệm của nhà nước và các biện pháp xử lý dữ liệu.",
 "answer": "Công ty cổ phần có quy định về bảo vệ dữ liệu người dùng, bao gồm việc đảm bảo dữ liệu được sử dụng một cách hợp pháp, bảo mật, và tuân thủ các quy định của pháp luật. Nhà nước có trách nhiệm giám sát và kiểm soát việc sử dụng dữ liệu, đặc biệt là dữ liệu cá nhân của người dùng. Các biện pháp xử lý dữ liệu được quy định trong Nghị định 109/2007 có các quy định về việc bảo vệ dữ liệu, bao gồm việc thiết lập các biện pháp bảo mật, kiểm soát truy cập, và xử lý dữ liệu theo nguyên tắc "chỉ khi cần, chỉ để cần, và không giữ lại dữ liệu ngoài phạm vi cần thiết."  Ngoài ra, công ty có quy định về việc xử lý dữ liệu theo quy định của pháp luật, đảm bảo dữ liệu được bảo vệ an toàn và không bị lạm dụng."
}
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/err

100%|██████████| 4142/4142 [5:08:20<00:00,  4.47s/it]


In [6]:
print(len(error_chunks))

0


In [9]:
from sklearn.model_selection import train_test_split
# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# Save the training and testing sets to CSV files
train_df.to_csv('data/train_data.csv', index=False)
test_df.to_csv('data/test_data.csv', index=False)
print("Train data saved to data/train_data.csv")
print("Test data saved to data/test_data.csv")

Train data saved to data/train_data.csv
Test data saved to data/test_data.csv
